<a href="https://colab.research.google.com/github/kavyajeetbora/ETL_wages/blob/master/Annual_Survey_of_Industries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
import pandas as pd
from zipfile import ZipFile
from glob import glob
from tqdm import tqdm
from lxml import etree

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Functions

In [67]:
def read_xls_file(filename):
    with open(filename, 'r') as xl_file:
        lines = xl_file.readlines()
        raw_html = "".join(lines)
        table_html = "".join(lines[1:])

    parser = etree.HTMLParser()
    html_root = etree.fromstring(raw_html, parser)
    table_root = etree.fromstring(table_html, parser)
    html_string = etree.tostring(table_root, pretty_print=True, method="html")

    return html_root, html_string

def extract_text(elems):
    elem_text = []
    for elem in elems:
        if elem is not None:
            if elem.text is not None:
                elem_text.append(elem.text)
    return elem_text


def reformat_table(df, html_root):

    industry_elems = html_root.findall(r'.//tbody[4]/td')
    industries = extract_text(industry_elems)

    state_elems = html_root.findall(r'.//tbody[5]/td')
    state = extract_text(state_elems)[0]

    variable_elems = html_root.findall(r'.//tbody[6]/td')
    variables = extract_text(variable_elems)
    variables = pd.Series(variables).unique()

    ## Extract the time duration
    years = df['Year'].copy().reset_index(drop=True)
    year_vals = years.loc[years.index.repeat(len(industries))].reset_index(drop=True)

    ## Now prepare the dataframe
    input_vals = df.iloc[:,1:].copy()
    input_vals = input_vals.values.ravel(order='C').reshape(-1,len(variables))
    input_vals = pd.DataFrame(input_vals)
    input_vals.columns = variables

    ind_vals = industries*len(years)

    #assign year and ind
    input_vals['year'] = year_vals
    input_vals["ind"] = ind_vals
    input_vals['state'] = state

    return input_vals

## Reformat Tables

Upload the XL file and replace the `filepath ` value with the filename

In [82]:
zip_file = '/gdrive/MyDrive/10states_allsectors_1980_2020.zip'
with ZipFile(zip_file, 'r') as myzip:
    myzip.extractall()

In [84]:
file_type = ['factories', 'gross', 'mandays', 'rent']
xl_files = glob('*.xls')

sheets = {}

for ftype in file_type:
    files = [f for f in xl_files if ftype in f]
    sheets[ftype] = files

In [85]:
def extract_tables(files):
    errors = []
    clean_tables = []

    for xl_file in tqdm(files, unit='.xls'):

        try:
            ## Parse html content
            html_root, html_string = read_xls_file(xl_file)

            ## Convert data to table
            df = pd.read_html(html_string)[0]

            ## Reformat the table
            reformatted_table = reformat_table(df, html_root)
            clean_tables.append(reformatted_table)

        except Exception as e:
            errors.append({"Error": e, "file": xl_file})
            print(e)
            clean_tables = None
            break

    clean_tables = pd.concat(clean_tables)
    return clean_tables

In [86]:
%%time
sheet_tables = {}
for k,v in sheets.items():
    table = extract_tables(v)
    sheet_tables[k] = table

100%|██████████| 40/40 [00:07<00:00,  5.26.xls/s]

CPU times: user 26.4 s, sys: 133 ms, total: 26.5 s
Wall time: 27 s


In [93]:
df_final.sample()

,Number of Factories,Fixed Capital,Working Capital,Physical Working Capital,Productive Capital,Invested Capital,Total Input,year,ind,state,Rent Paid
1018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021 - 2022,324 - manufacture of games and toys,Kerala,NaN


In [120]:
df_final = sheet_tables[list(sheet_tables.keys())[0]]
for k,v in sheet_tables.items():
    if k!= list(sheet_tables.keys())[0]:
        df_final = pd.merge(
            left=df_final,
            right=v,
            on=['year', 'ind', 'state'],
            how='left'
        )

df_final['Rent Paid'] = df_final['Rent Paid_x'] + df_final['Rent Paid_y']
df_final = df_final.drop(['Rent Paid_x', 'Rent Paid_y'], axis=1)

In [122]:
index_columns = ['year','state', 'ind']
variable_columns = [c for c in df_final.columns if c not in index_columns]
index_columns.extend(variable_columns)
df_final = df_final[index_columns]

In [123]:
df_final.sample(5)

,year,state,ind,Number of Factories,Fixed Capital,Working Capital,Physical Working Capital,Productive Capital,Invested Capital,Total Input,...,Wages and Salaries - Workers,Wages and Salaries - Supervisory and Managerial Staff,Number of Employees,Interest Paid,Depreciation,Net Income,Net Value Added,Gross Value Added,Net Fixed Capital Formation,Rent Paid
16543,1997 - 1998,Assam,381 - waste collection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10511,2016 - 2017,Madhya Pradesh,282 - manufacture of special-purpose machinery,121.0,70828.0,73668.0,69693.0,144496.0,140520.0,363175.0,...,5604.0,15906.0,6409.0,1826.0,7715.0,85075.0,87346.0,95061.0,2462.0,NaN
26699,1982 - 1983,Karnataka,329 - other manufacturing n.e.c.,26.0,47.0,73.0,62.0,120.0,109.0,156.0,...,15.0,NaN,421.0,11.0,7.0,34.0,49.0,57.0,3.0,NaN
22636,2013 - 2014,Gujarat,203 - manufacture of man-made fibres,25.0,186536.0,42818.0,52732.0,229354.0,239267.0,302781.0,...,9221.0,4758.0,6661.0,10237.0,15169.0,41002.0,50049.0,65218.0,7374.0,NaN
21817,2018 - 2019,Madhya Pradesh,383 - materials recovery,18.0,3052.0,756.0,774.0,3808.0,3826.0,4525.0,...,117.0,150.0,178.0,123.0,334.0,1042.0,1168.0,1502.0,1574.0,NaN


In [124]:
df_final.to_excel('Annual Survey of Industries.xlsx', index=False)